# Neural Network to Predict Temperature
This notebook demonstrates a neural network predicting the temperature for the last timestamp based on historical data.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Load and Clean the Data

In [ ]:
# Load the dataset
file_path = '/mnt/data/dataexport_20241201T172719.xlsx'  # Update this if necessary
data = pd.read_excel(file_path)

# Extract relevant data: Timestamps and temperature values
cleaned_data = data.iloc[9:].copy()
cleaned_data.columns = ['timestamp', 'temperature']  # Rename columns for clarity
cleaned_data['temperature'] = pd.to_numeric(cleaned_data['temperature'], errors='coerce')  # Ensure numeric
cleaned_data['timestamp'] = pd.to_datetime(cleaned_data['timestamp'], errors='coerce')  # Ensure datetime
cleaned_data = cleaned_data.dropna().reset_index(drop=True)  # Drop missing rows and reset index

cleaned_data.head()

### Prepare Data for Training and Testing

In [ ]:
# Use all rows except the last one for training
train_data = cleaned_data.iloc[:-1]
test_data = cleaned_data.iloc[-1:]

# Extract input (time as hours) and output (temperature)
X_train = (train_data['timestamp'].dt.hour.values).reshape(-1, 1)
y_train = train_data['temperature'].values

# Prepare the test input
X_test = np.array([[test_data['timestamp'].dt.hour.values[0]]])
y_test_actual = test_data['temperature'].values[0]  # Actual temperature for comparison

# Normalize the input (time divided by 24)
X_train_norm = X_train / 24
X_test_norm = X_test / 24

### Build and Train the Neural Network

In [ ]:
# Create the neural network
model = Sequential([
    Dense(10, activation='relu', input_shape=(1,)),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train_norm, y_train, epochs=100, verbose=0)

### Make Predictions

In [ ]:
# Predict the temperature for the last timestamp
predicted_temp = model.predict(X_test_norm)

# Display the actual and predicted temperature
print(f"Actual temperature: {y_test_actual}°C")
print(f"Predicted temperature: {predicted_temp[0][0]:.2f}°C")